In [ ]:
# default_exp projects

# Projects

Tight.ai projects are essentially your ML models as production-ready cloud services. 

Projects are serverless machine learning applications as REST API endpoints made for scalability, high availability, and improvement simplicity.

Serverless means you pay for what you use, when you use it thus drastically reducing the cost of every serverless app you have in production.

In [ ]:
#hide
# !pip install python-slugify

In [ ]:
#export
import time
import urllib.parse
import webbrowser
from pathlib import Path
from slugify import slugify
from tightai import exceptions
from tightai.exceptions import ArgumentsRequired
from tightai.lookup import Lookup
from tightai.base import BaseObject, base_tightignore
from tightai.conf import CLI_ENDPOINT
from tightai.user import UserObject
from tightai.upload import Uploader
from tightai.utils import sort_by_key, print_dict

In [ ]:
#hide
test = True
if test:
    CLI_ENDPOINT = "http://cli.desalsa.io:8000"

In [ ]:
#export
class VersionManager(Lookup, BaseObject):
    api = CLI_ENDPOINT
    version = None
    project_id = None
    
    def __str__(self):
        return f"<VersionManager>"
    
    def __repr__(self):
        return self.__str__()
    
    def create(self, project_id=None, raw=False, **kwargs):
        if project_id == None:
            raise ArgumentsRequired("project_id")
        r = self.http_post(f"/projects/{project_id}/versions/", data=kwargs)
        if raw:
            return r
        self.handle_invalid_lookup(r, expected_status_code=201)
        return Version(**r.json())
    
    def get(self, version=None, project_id=None, raw=False):
        if project_id == None or version == None:
            raise ArgumentsRequired("version", "project_id")
        r = self.http_get(f"/projects/{project_id}/versions/{version}")
        if raw:
            return r
        self.handle_invalid_lookup(r, expected_status_code=200)
        return Version(**r.json())
    
    def latest(self, project_id=None, none_on_404=False, deployed=False):
        if project_id == None:
            raise ArgumentsRequired("project_id")
        deployed_str = f"{deployed}".lower()
        r = self.http_get(f"/projects/{project_id}/latest/?deployed={deployed_str}")
        if deployed == True:
            none_on_404 = True
        if r.status_code == 404 and none_on_404:
            return None
        self.handle_invalid_lookup(r, expected_status_code=200)
        return Version(**r.json())
    
    def all(self, project_id=None, **kwargs):
        if project_id == None:
            raise ArgumentsRequired("project_id")
        endpoint = f"/projects/{project_id}/versions/"
        if len(kwargs.keys()):
            query_string = urllib.parse.urlencode(kwargs)
            endpoint = f"{endpoint}?{query_string}"            
        r = self.http_get(endpoint)
        self.handle_invalid_lookup(r, expected_status_code=200)
        return [Version(**x) for x in r.json()['results']]

    def get_or_create(self, **kwargs):
        created = False
        version = None
        try:
            existing_r = self.get(**kwargs)
        except:
            existing_r = None
        if existing_r != None:
            return existing_r, created
        new_v = self.create(**kwargs)
        created = True
        return new_v, created
    

class Version(Lookup, BaseObject):
    version = None
    project_id = None
    trigger_options = ['status', 'build', 'build_and_deploy', 'deploy', 'destroy']
    display_options = ['project_id', 'version', 'description', 'url', 'deployed', 
                       'deployed_timestamp', 'destroyed', 'destroyed_timestamp', 'updated', 'timestamp']
    
    def __str__(self):
        return f"<Version {self.version} ({self.project_id})>"
    
    objects = VersionManager()
    
    def build(self):
        return self.update_trigger(option='build')
    
    def delete(self):
        confirmation_code = f"{self.project_id}-v{self.version}"
        response = input(f"Deleting is cannot be reversed. Please confirm by typing:\n\n{confirmation_code}\n\n")
        if f"{response}".strip() != confirmation_code:
            print(f"\nDelete failed. {response} did not match {confirmation_code} exactly.")
            return 
        r = self.http_delete(f"/projects/{self.project_id}/versions/{self.version}")
        self.handle_invalid_lookup(r, expected_status_code=204)
        setattr(self, 'deleted', True)
        return r
    
    def summary(self, verbose=0):
        data = {
            "version": self.version,
            "project_id": self.project_id,
            "url": self.url,
            "deployed": self.deployed,
            "updated": self.updated
        }
        if verbose == 1:
            print_dict(data)
        return data
    
    def deploy(self):
        raise Exception("Use `.push(local_path)` to deploy instead.")
    
    def destroy(self):
        return self.update_trigger(option='destroy')
    
    def upload(self, path, verbose=True):
        src_path = Path(path)
        tightai_ignore = src_path / ".tightignore"
        if not tightai_ignore.exists():
            print("No .tightignore found. Creating default.")
            with open(str(tightai_ignore), "w") as f:
                f.writelines(base_tightignore)
        uploader = Uploader(path=path, project_id=self.project_id, version=self.version)
        uploaded_count = uploader.upload(verbose=verbose)
        return uploaded_count
    
    def status(self, wait=True, display=True):
        endpoint = f"/projects/{self.project_id}/versions/{self.version}/status/"
        r = self.http_post(endpoint, data={})
        if r.status_code != 200:
            raise Exception("Could not find status at this time.")
        r_data = r.json()
        for k,v in r_data.items():
            setattr(self, k, v)
        verbose = 1
        if display == False:
            verbose = 0
        return self.details(verbose=verbose)
    
    def refresh(self):
        version = self.objects.get(version=self.version, project_id=self.project_id)
        for k,v in version.__dict__.items():
            setattr(self, k, v)
        return version.details()
    
    def push(self, path, check_status=True):
        project_id = self.project_id
        version = self.version
        uploaded_count = self.upload(path, verbose=True)
        if uploaded_count == 0:
            print(f"Uploaded 0 files. Continuing")
        print("Deploying...")
        deploy_r = self.update_trigger(option='build_and_deploy')
        if deploy_r.status_code != 200:
            raise Exception("Project not deployed. Please try again.")
        if not check_status:
            print(f"{project_id} {version} pushed. Use `.status()` to check deployment status.")
            return None
        details = self.status(wait=True)
        url = details.get('url')
        if not url:
            print("URL is not ready yet. Use `.status()` to check deployment status.")
            return None
        setattr(self, 'url', url)
        return url
    
    def update_trigger(self, option='status'):
        if option not in self.trigger_options:
            print(f"{option} is invalid. Please try again.")
            option = 'status'
        data = {
            'trigger_active': True,
            'trigger': f"{option}",
        }
        r = self.http_put(f"/projects/{self.project_id}/versions/{self.version}", data=data)
        self.handle_invalid_lookup(r, expected_status_code=200)
        self.update_from_response(r)
        return r
    
    def update_from_response(self, response, display=False):
        for k, v in response.json().items():
            if display:
                print(f"{k}")
            current_v = getattr(self, k)
            if v != None and current_v != v:
                setattr(self, k, v)
        return
    
    def open(self):
        details = self.details()
        url = details.get("url")
        if url == None or details['deployed'] == False:
            raise Exception("This version has not been deployed yet. Push a deployment or refresh version status.")
        return webbrowser.open(url)
        

In [ ]:
#export
class ProjectManager(Lookup, BaseObject):
    api = CLI_ENDPOINT
    version = None
    
    def __str__(self):
        return f"<ProjectManager>"
    
    def __repr__(self):
        return self.__str__()
        
    def create(self, **kwargs):
        r = self.http_post(f"/projects/", data=kwargs)
        self.handle_invalid_lookup(r, expected_status_code=201)
        return Project(**r.json())
    
    def get(self, project_id=None):
        if project_id == None:
            raise ArgumentsRequired("project_id")
        r = self.http_get(f"/projects/{project_id}/")
        self.handle_invalid_lookup(r, expected_status_code=200)
        return Project(**r.json())
    
    def all(self, **kwargs):
        endpoint = f"/projects/"
        if len(kwargs.keys()):
            query_string = urllib.parse.urlencode(kwargs)
            endpoint = f"{endpoint}?{query_string}"            
        r = self.http_get(endpoint)
        self.handle_invalid_lookup(r, expected_status_code=200)
        return [Project(**x) for x in r.json()['results']]

    def get_or_create(self, **kwargs):
        created = False
        existing_r = self.get()
        if existing_r.status_code == 200:
            return existing_r, created
        new_r = self.create(**kwargs)
        self.handle_invalid_lookup(new_r, expected_status_code=201)
        created = True
        return new_r, created
    

class Project(Lookup, BaseObject):
    project_id = None
    display_options = ['name', 'id', 'description', 
                       'updated', 'timestamp', 'versions_metadata', 'versions_urls']
    
    def __str__(self):
        return f"<Project {self.id}>"
    
    def summary(self, verbose=0):
        data = {
            "name": self.name,
            "project_id": self.project_id,
            "versions_summary": {
                "deployed": self.versions_metadata.get('deployed_count'),
                "urls": self.versions_urls,
            },
            "updated": self.updated
        }
        if verbose == 1:
            print_dict(data)
        return data
    
    @property
    def project_id(self):
        return self.id
    
    objects = ProjectManager()
    
    def create_version(self, version=None):
        if version == None:
            raise ArgumentsRequired("version")
        version, _ = Version.objects.get_or_create(project_id=self.project_id, version=version)
        return version
    
    def get_version(self, version=None):
        if version == None:
            raise ArgumentsRequired("version")
        version, _ = Version.objects.get_or_create(project_id=self.project_id, version=version)
        return version
    
    def delete(self):
        print("You can only delete projects in the tight.ai console right now.")
    
    def get_versions(self):
        versions = Version.objects.all(project_id=self.project_id)
        return versions
    
    def latest(self, deployed=False):
        return Version.objects.latest(project_id=self.project_id, deployed=deployed)

In [ ]:
#hide
proj = Project.objects.get(project_id='news-categories')
proj.project_id

RequestError: 'Invalid token. (status 401)'

In [ ]:
#hide
proj.summary()

NameError: name 'proj' is not defined

In [ ]:
#hide
proj.details(verbose=1)

NameError: name 'proj' is not defined

In [ ]:
#hide
proj.get_versions()

In [ ]:
#hide
recent_version = proj.latest()
if recent_version != None:
    recent_version.summary(verbose=1)
    print("\n\n")
    recent_version.details()

In [ ]:
#hide
recent_version.open()

In [ ]:
#hide
version_c2, created = Version.objects.get_or_create(version=92023, project_id="news-categories")
version_c2
print(version_c2, created)
time.sleep(4)
version_c2.delete()
print('deleted')

In [ ]:
#hide
try:
    version_c = Version.objects.get(version=92023, project_id="news-categories")
except Exception as e:
    print("Expetected Error:", e.message)
    assert e.status == 404

In [ ]:
#hide
try:
    version_c = Version.objects.create(version=99023, project_id="news-categories")
except Exception as e:
    print("Expetected Error:", e.message)
    assert e.status == 400

In [ ]:
#hide
version = Version.objects.get(version=99023, project_id="news-categories")
version.details()

In [ ]:
#hide
version = Version.objects.get(version=6, project_id="news-categories")
# version.refresh()

In [ ]:
#hide
deets = version.details()
import json
json.dumps(deets)

In [ ]:
#hide
version.push("/Users/jmitch/tight/my-tight-apps/local_dev/")

In [ ]:
#hide
version.details()

In [ ]:
#hide
version.status()

In [ ]:
#hide
# version.destroy()

In [ ]:
#hide
version.status()

In [ ]:
#hide
versions = Version.objects.all(project_id="news-categories", size='10', deployed=True)
versions

In [ ]:
#hide
# import requests
# r = requests.post('https://www.tight.ai/api/webhooks/projects/versions/status/', data={})

In [ ]:
#hide
d = "{\"app_name\": \"news-categories-v5\", \"address\": {\"url\": \"http://news-categories-v5.apps.svc.cluster.local\"}, \"conditions\": [{\"lastTransitionTime\": \"2020-05-19T03:48:44Z\", \"status\": \"True\", \"type\": \"ConfigurationsReady\"}, {\"lastTransitionTime\": \"2020-05-19T03:48:47Z\", \"status\": \"True\", \"type\": \"Ready\"}, {\"lastTransitionTime\": \"2020-05-19T03:48:47Z\", \"status\": \"True\", \"type\": \"RoutesReady\"}], \"latestCreatedRevisionName\": \"news-categories-v5-rk6bb\", \"latestReadyRevisionName\": \"news-categories-v5-rk6bb\", \"observedGeneration\": 1, \"traffic\": [{\"latestRevision\": true, \"percent\": 100, \"revisionName\": \"news-categories-v5-rk6bb\"}], \"url\": \"http://news-categories-v5.apps.tight.ml\"}"

In [ ]:
#hide
import json

In [ ]:
#hide
b = json.loads(d)

In [ ]:
#hide
b

In [ ]:
#hide
import requests
endpoint = "http://127.0.0.1:8123/status"
data = {
    'project_id': "news-categories",
    "version": "6"
}
r = requests.post(endpoint, json=data)
data = r.json()
data

In [ ]:
#hide
import requests
endpoint = "http://127.0.0.1:8123/live/status/news-categories/v5"
r = requests.get(endpoint)
data = r.json()
data

In [ ]:
#hide
traffic_data = data['status']['traffic']
traffic_data = sorted(traffic_data, key = lambda x: -x['percent'])
traffic_data

In [ ]:
#hide
ready = False
for i in data['status']['conditions']:
    _cond_ready = f"{i['status']}".lower() == "true"
    ready = _cond_ready
ready

In [ ]:
#hide
sorted(traffic_data, key = lambda x: -x['percent'])